In [23]:
import pandas as pd, numpy as np
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [58]:
sales, forecast = pd.read_excel('/Users/glebsokolov/Downloads/Тестовое задание.xlsx',
                                sheet_name='продажи'), pd.read_excel(
    '/Users/glebsokolov/Downloads/Тестовое задание.xlsx', sheet_name='прогноз')
sales = sales.rename({sales.columns[-1]: 'sales', sales.columns[2]: 'week', 'ТК': 'tc', 'Товар': 'item'}, axis=1)
forecast = forecast.rename({forecast.columns[-1]: 'forecast'}, axis=1)
sales = sales.join(forecast.forecast, how='left')
sales['error'] = np.abs(sales.sales - forecast.forecast)

In [59]:
sales.columns

Index(['item', 'tc', 'week', 'sales', 'forecast', 'error'], dtype='object')

In [60]:
sales.set_index(['tc', 'week'])


item  sales  forecast  error
tc week                              
S1 14    4452    158       123     35
   14    7346    110       195     85
   14    7852    199       199      0
   14    1056    113       174     61
   14    5011    132       135      3
...       ...    ...       ...    ...
S3 17    2339    193       146     47
   17    1219    161       139     22
   17    7925    169       176      7
   17    3659    121       154     33
   17    3729    164       101     63

[1188 rows x 4 columns]

In [110]:
wape = {}
wape_max = {}
for category in set(sales['tc'].to_numpy()):
    cat = sales[sales['tc'] == category].set_index('week').sort_index(ascending=True)
    wv, wvm = {}, {}
    for week in sorted(set(cat.index.to_numpy())):
        week_data = cat[cat.index.isin([week])]
        max_sales = max(week_data.sales)
        wv[week] = sum(week_data.error.to_numpy()) / sum(week_data.sales.to_numpy())
        wvm[week] = 2*0.99 * sum(
            week_data[week_data['forecast'] >= max_sales]['forecast'].to_numpy() - max_sales) + 0.01 * sum(
            max_sales - week_data[week_data['forecast'] < max_sales]['forecast'].to_numpy()) / sum(
            week_data.sales.to_numpy())
    wape[category] = wv
    wape_max[category] = wvm
wape


{'S2': {14: 0.22464573492636844,
  15: 0.2293145776657156,
  16: 0.2213215258855586,
  17: 0.21420889904557064},
 'S1': {14: 0.20360489328474754,
  15: 0.23930508823731375,
  16: 0.19571844725967075,
  17: 0.23810176930462212},
 'S3': {14: 0.2366107654855288,
  15: 0.20784574468085107,
  16: 0.22069341325522784,
  17: 0.23442387904066736}}

In [111]:
wape_max

{'S2': {14: 7.923139761044735,
  15: 0.003181607059916396,
  16: 0.003166893732970027,
  17: 0.0034030111574136312},
 'S1': {14: 0.003178682977615825,
  15: 0.0034690654398132257,
  16: 1.9833696904003795,
  17: 15.843173090111096},
 'S3': {14: 0.0033912631863673246,
  15: 0.003339095744680851,
  16: 5.9432797493358756,
  17: 0.0029412148070907195}}